## Cart-Pole Balancing using DQNs
Objective: balance a cartpole using deep learning, specifically the two core pieces of DQNs, the epsilon greedy algorithm and memory replay. 

Goal: try to achive a score of 100 or above.

In [1]:
#!pip3 install --user gym gym[atari]

In [2]:
from collections import deque
import numpy as np
import random

import gym
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95   # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(state_size,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_size, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=0.001))
        return self.model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))


    def act(self, state):
        
        if np.random.rand() <= self.epsilon:
            return random.randrange(action_size)
    
        
        else:
            q_values = self.model.predict(state)
            return np.argmax(q_values[0])
            

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
            
        # Decay the epsilon value 
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [ ]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    
    agent = DQNAgent(state_size, action_size)
    done = False
    batch_size = 32 # Try various
    EPISODES = 100   # Number of episodes to play

    
    for eps in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            
            agent.remember(state, action, reward, next_state, done)
            

            state = next_state
            if done:
                print("episode: {}/{}, score: {}, eps: {:.2}".format(eps, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        if eps % 10 == 0:
            agent.save("./cartpole-dqn.h5")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0/100, score: 10, eps: 1.0
episode: 1/100, score: 8, eps: 1.0
episode: 2/100, score: 59, eps: 0.79
episode: 3/100, score: 21, eps: 0.71
episode: 4/100, score: 12, eps: 0.67
episode: 5/100, score: 8, eps: 0.64
episode: 6/100, score: 13, eps: 0.6
episode: 7/100, score: 16, eps: 0.56
episode: 8/100, score: 14, eps: 0.52
episode: 9/100, score: 9, eps: 0.5
episode: 10/100, score: 8, eps: 0.48
episode: 11/100, score: 19, eps: 0.43
episode: 12/100, score: 11, eps: 0.41
episode: 13/100, score: 14, eps: 0.38
episode: 14/100, score: 10, eps: 0.36
episode: 15/100, score: 11, eps: 0.34
episode: 16/100, score: 11, eps: 0.33
episode: 17/100, score: 15, eps: 0.3
episode: 18/100, score: 68, eps: 0.21
episode: 19/100, score: 9, eps: 0.21
episode: 20/100, score: 14, eps: 0.19
episode: 21/100, score: 29, eps: 0.17


The agent peforms poorly at first but performance dramatically increases around episode 37. The highest score obtained after 100 episodes was 499.